In [5]:
import psycopg2
import json
from typing import Dict
import os
from dotenv import load_dotenv

# Lade Umgebungsvariablen
load_dotenv()


def connect_to_db() -> psycopg2.extensions.connection:
    return psycopg2.connect(
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT"),
        database=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
    )


def get_category_counts(cur: psycopg2.extensions.cursor) -> Dict[str, int]:
    """Ermittelt die Anzahl der verfügbaren Einträge pro DDC-Kategorie"""
    cur.execute(
        """
        SELECT SUBSTRING(ddc FROM 1 FOR 1) as ddc_category, COUNT(*) as count
        FROM dnb_records
        WHERE num_pages <= 200 
            AND abstract_num IS NOT NULL 
            AND abstract_num != '0'
            AND id NOT IN (SELECT id FROM dnb_records_subset)
        GROUP BY ddc_category
        ORDER BY ddc_category
    """
    )
    return {str(row[0]): row[1] for row in cur.fetchall()}


def create_balanced_subset(category_counts: Dict[str, int], total_needed: int) -> Dict[str, int]:
    """
    Erstellt ein ausgewogenes Subset der Daten durch rotierende Auswahl.
    Verteilt die Einträge gleichmäßig auf alle Kategorien, die noch Einträge haben.
    """
    entries_needed = {ddc: 0 for ddc in category_counts.keys()}
    remaining_entries = {ddc: count for ddc, count in category_counts.items()}
    entries_to_allocate = total_needed

    while entries_to_allocate > 0:
        # Nur Kategorien berücksichtigen, die noch Einträge haben
        available_categories = [ddc for ddc, count in remaining_entries.items() if count > 0]

        if not available_categories:
            break

        # Berechne, wie viele Einträge pro Kategorie in dieser Runde verteilt werden
        entries_per_category = max(1, entries_to_allocate // len(available_categories))

        for ddc in available_categories:
            # Nimm den kleineren Wert: verfügbare Einträge oder zu verteilende Einträge
            entries_to_take = min(remaining_entries[ddc], entries_per_category, entries_to_allocate)

            entries_needed[ddc] += entries_to_take
            remaining_entries[ddc] -= entries_to_take
            entries_to_allocate -= entries_to_take

            if entries_to_allocate <= 0:
                break

    return entries_needed


def extend_subset_table(cur: psycopg2.extensions.cursor, additional_entries: int) -> None:
    """Erweitert die bestehende Subset-Tabelle um weitere Einträge"""
    # Aktuelle Verteilung ermitteln und anzeigen
    cur.execute(
        """
        SELECT SUBSTRING(ddc FROM 1 FOR 1) as ddc_category, COUNT(*) as count
        FROM dnb_records_subset
        GROUP BY ddc_category
        ORDER BY ddc_category
    """
    )
    print("Aktuelle Verteilung:")
    for row in cur.fetchall():
        print(f"DDC {row[0]}: {row[1]} Einträge")

    # Neue Verteilung berechnen
    category_counts = get_category_counts(cur)
    additional_needed = create_balanced_subset(category_counts, additional_entries)

    print("\nGeplante zusätzliche Einträge:")
    for cat, count in sorted(additional_needed.items()):
        print(f"DDC {cat}: +{count} Einträge")

    # Zusätzliche Einträge einfügen
    cur.execute(
        """
        INSERT INTO dnb_records_subset
        WITH ranked_records AS (
            SELECT *,
                ROW_NUMBER() OVER (
                    PARTITION BY SUBSTRING(ddc FROM 1 FOR 1)
                    ORDER BY RANDOM()
                ) as row_num
            FROM dnb_records r
            WHERE num_pages <= 200 
                AND abstract_num IS NOT NULL 
                AND abstract_num != '0'
                AND NOT EXISTS (
                    SELECT 1 FROM dnb_records_subset s
                    WHERE s.id = r.id
                )
        )
        SELECT * FROM ranked_records r
        WHERE row_num <= (
            SELECT count::integer
            FROM jsonb_each_text(%s) as t(ddc, count)
            WHERE t.ddc = SUBSTRING(r.ddc FROM 1 FOR 1)
        )
    """,
        (json.dumps(additional_needed),),
    )


def main():
    conn = connect_to_db()
    cur = conn.cursor()

    try:
        print("Erweitere Subset-Tabelle...")
        extend_subset_table(cur, 80)

        # Überprüfe das erweiterte Subset
        cur.execute(
            """
            SELECT COUNT(*) FROM dnb_records_subset
        """
        )
        total_count = cur.fetchone()[0]
        print(f"\nNeue Gesamtanzahl: {total_count}")

        # Neue Verteilung überprüfen
        cur.execute(
            """
            SELECT SUBSTRING(ddc FROM 1 FOR 1) as ddc_category, COUNT(*) as count
            FROM dnb_records_subset
            GROUP BY ddc_category
            ORDER BY ddc_category
        """
        )
        print("\nNeue Verteilung:")
        for row in cur.fetchall():
            print(f"DDC {row[0]}: {row[1]} Einträge")

        conn.commit()
        print("\nErweiterung erfolgreich abgeschlossen!")

    finally:
        cur.close()
        conn.close()


if __name__ == "__main__":
    main()

Erweitere Subset-Tabelle...
Aktuelle Verteilung:
DDC  : 2 Einträge
DDC 0: 3255 Einträge
DDC 1: 1154 Einträge
DDC 2: 25 Einträge
DDC 3: 3255 Einträge
DDC 4: 232 Einträge
DDC 5: 3254 Einträge
DDC 6: 3254 Einträge
DDC 7: 414 Einträge
DDC 8: 45 Einträge
DDC 9: 84 Einträge

Geplante zusätzliche Einträge:
DDC 0: +16 Einträge
DDC 3: +16 Einträge
DDC 5: +16 Einträge
DDC 6: +16 Einträge
DDC None: +16 Einträge

Neue Gesamtanzahl: 15038

Neue Verteilung:
DDC  : 2 Einträge
DDC 0: 3271 Einträge
DDC 1: 1154 Einträge
DDC 2: 25 Einträge
DDC 3: 3271 Einträge
DDC 4: 232 Einträge
DDC 5: 3270 Einträge
DDC 6: 3270 Einträge
DDC 7: 414 Einträge
DDC 8: 45 Einträge
DDC 9: 84 Einträge

Erweiterung erfolgreich abgeschlossen!


In [1]:
import psycopg2
import os
from dotenv import load_dotenv
from datetime import datetime

load_dotenv()


def connect_to_db() -> psycopg2.extensions.connection:
    return psycopg2.connect(
        dbname=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT"),
    )


def create_backup():
    conn = connect_to_db()
    cur = conn.cursor()

    try:
        # Backup erstellen
        print("Erstelle Backup...")
        cur.execute(
            """
            CREATE TABLE dnb_records_subset_backup AS 
            SELECT * FROM dnb_records_subset;
        """
        )

        # Überprüfe die Anzahl der Einträge
        cur.execute("SELECT COUNT(*) FROM dnb_records_subset")
        original_count = cur.fetchone()[0]

        cur.execute("SELECT COUNT(*) FROM dnb_records_subset_backup")
        backup_count = cur.fetchone()[0]

        print(f"\nOriginal Einträge: {original_count}")
        print(f"Backup Einträge: {backup_count}")

        if original_count == backup_count:
            print("\nBackup erfolgreich erstellt!")

            # Zeige Verteilung in beiden Tabellen
            print("\nÜberprüfe DDC-Verteilung:")
            for table in ["dnb_records_subset", "dnb_records_subset_backup"]:
                cur.execute(
                    f"""
                    SELECT SUBSTRING(ddc FROM 1 FOR 1) as ddc_category, COUNT(*) as count
                    FROM {table}
                    GROUP BY ddc_category
                    ORDER BY ddc_category
                """
                )
                print(f"\n{table}:")
                for row in cur.fetchall():
                    print(f"DDC {row[0]}: {row[1]} Einträge")

            conn.commit()
        else:
            print("\nWARNUNG: Anzahl der Einträge stimmt nicht überein!")
            conn.rollback()

    except Exception as e:
        print(f"Fehler beim Backup: {e}")
        conn.rollback()
    finally:
        cur.close()
        conn.close()


if __name__ == "__main__":
    create_backup()

Erstelle Backup...

Original Einträge: 10000
Backup Einträge: 10000

Backup erfolgreich erstellt!

Überprüfe DDC-Verteilung:

dnb_records_subset:
DDC 0: 2012 Einträge
DDC 1: 1154 Einträge
DDC 2: 25 Einträge
DDC 3: 2012 Einträge
DDC 4: 232 Einträge
DDC 5: 2011 Einträge
DDC 6: 2011 Einträge
DDC 7: 414 Einträge
DDC 8: 45 Einträge
DDC 9: 84 Einträge

dnb_records_subset_backup:
DDC 0: 2012 Einträge
DDC 1: 1154 Einträge
DDC 2: 25 Einträge
DDC 3: 2012 Einträge
DDC 4: 232 Einträge
DDC 5: 2011 Einträge
DDC 6: 2011 Einträge
DDC 7: 414 Einträge
DDC 8: 45 Einträge
DDC 9: 84 Einträge
